# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe088620f40>
├── 'a' --> tensor([[-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134]])
└── 'x' --> <FastTreeValue 0x7fe088620f10>
    └── 'c' --> tensor([[-1.5627, -0.5615,  0.0244, -0.9923],
                        [-0.3506,  0.9833,  0.0886, -1.1612],
                        [-0.2403, -1.5833, -1.2538, -0.0843]])

In [4]:
t.a

tensor([[-0.7819,  1.6581,  0.9676],
        [-0.2390,  0.5912,  0.3134]])

In [5]:
%timeit t.a

61.3 ns ± 0.0462 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe088620f40>
├── 'a' --> tensor([[ 0.1226,  0.2744,  0.6054],
│                   [-2.6054,  0.6436, -0.8298]])
└── 'x' --> <FastTreeValue 0x7fe088620f10>
    └── 'c' --> tensor([[-1.5627, -0.5615,  0.0244, -0.9923],
                        [-0.3506,  0.9833,  0.0886, -1.1612],
                        [-0.2403, -1.5833, -1.2538, -0.0843]])

In [7]:
%timeit t.a = new_value

60 ns ± 0.0218 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7819,  1.6581,  0.9676],
               [-0.2390,  0.5912,  0.3134]]),
    x: Batch(
           c: tensor([[-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843]]),
       ),
)

In [10]:
b.a

tensor([[-0.7819,  1.6581,  0.9676],
        [-0.2390,  0.5912,  0.3134]])

In [11]:
%timeit b.a

56.5 ns ± 0.00925 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0066,  0.0404,  0.0589],
               [ 0.4824,  1.1163,  0.1291]]),
    x: Batch(
           c: tensor([[-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.0945 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.295 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 738 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe0886174f0>
├── 'a' --> tensor([[[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]],
│           
│                   [[-0.7819,  1.6581,  0.9676],
│                    [-0.2390,  0.5912,  0.3134]]])
└── 'x' --> <FastTreeValue 0x7fe088617bb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 55.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fdfdd30ee50>
├── 'a' --> tensor([[-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134],
│                   [-0.7819,  1.6581,  0.9676],
│                   [-0.2390,  0.5912,  0.3134]])
└── 'x' --> <FastTreeValue 0x7fdfdd30ef40>
    └── 'c' --> tensor([[-1.5627, -0.5615,  0.0244, -0.9923],
                        [-0.3506,  0.9833,  0.0886, -1.1612],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.5627, -0.5615,  0.0244, -0.9923],
                       [-0.3506,  0.9833,  0.0886, -1.1612],
                       [-0.2403, -1.5833, -1.2538, -0.0843]],
              
                      [[-1.5627, -0.5615,  0.0244, -0.9923],
                       [-0.3506,  0.9833,  0.0886, -1.1612],
                       [-0.2403, -1.5833, -1.2538, -0.0843]],
              
                      [[-1.5627, -0.5615,  0.0244, -0.9923],
                       [-0.3506,  0.9833,  0.0886, -1.1612],
                       [-0.2403, -1.5833, -1.2538, -0.0843]],
              
                      [[-1.5627, -0.5615,  0.0244, -0.9923],
                       [-0.3506,  0.9833,  0.0886, -1.1612],
                       [-0.2403, -1.5833, -1.2538, -0.0843]],
              
                      [[-1.5627, -0.5615,  0.0244, -0.9923],
                       [-0.3506,  0.9833,  0.0886, -1.1612],
                       [-0.2403, -1.5833, -1.2538, -0.0843]],

In [26]:
%timeit Batch.stack(batches)

77.2 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843],
                      [-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843],
                      [-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843],
                      [-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843],
                      [-1.5627, -0.5615,  0.0244, -0.9923],
                      [-0.3506,  0.9833,  0.0886, -1.1612],
                      [-0.2403, -1.5833, -1.2538, -0.0843],
                      [-1.5627, -0.5615,  0.0244, -0.9923],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
